<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

**1a.** Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

In [ ]:
# ZIP: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
# README.int.txt: https://www.d.umn.edu/~tpederse/Data/README.int.txt

**1b.** Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?


In [ ]:
# Le fichier est un corpus composé de 2369 phrases extraites du Wall Street Journal (Penn Treebank).
# Chaque phrase est séparée par une ligne contenant uniquement $$.
# Chaque phrase contient exactement une occurrence annotée du mot interest ou interests (au pluriel).
# Les sens sont annotés manuellement selon 6 sens définis par le Longman Dictionary of Contemporary English (LDOCE) :
# interest_1 : attention ou curiosité
# interest_2 : qualité de susciter l'attention
# interest_3 : activité à laquelle on porte attention
# interest_4 : avantage ou intérêt personnel
# interest_5 : part d'une entreprise ou affaire
# interest_6 : intérêt au sens financier (argent)
# L’annotation suit ce format : interest_6/NN (mot + numéro de sens + étiquette morpho-syntaxique POS)

**1c.** Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?


In [ ]:
# Oui. Le corpus contient des occurrences au pluriel annotées de la même façon : interests_5/NNS, interests_4/NNS, etc.

**1d.** Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

In [ ]:
# Chaque occurrence unique dans une phrase est annotée individuellement avec son propre numéro de sens (interest_1, interest_5, etc.).
# Ces phrases restent dans une seule ligne séparée par $$ comme les autres.

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [ ]:
# Veuillez répondre ici (en commentaire) à la question.
# Sense 1 =  361 occurrences (15%) - readiness to give attention  
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to  
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to  
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor  
# Sense 5 =  500 occurrences (21%) - a share in a company or business  
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money  

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [ ]:
# Veuillez répondre ici (en commentaire) à la question.
# Les sens viennent du Longman Dictionary of Contemporary English (LDOCE).
# Il est mentionné dans le README : « the six non-idiomatic noun senses of "interest" defined in the electronic version of the first edition of Longman's Dictionary of Contemporary English ».
# On peut consulter une version en ligne ici : https://www.ldoceonline.com/

In [ ]:
# Alignement des définitions du dictionnaire (exemples reformatés) :
#    Sense 1 = "readiness to give attention"
#        e.g., "He listened with great interest."
#    Sense 2 = "quality of causing attention to be given to"
#        e.g., "The story is of little interest."
#    Sense 3 = "an activity that you enjoy doing or subject that you enjoy studying"
#        e.g., "My main interests are photography and hiking."
#    Sense 4 = "advantage, benefit or self-serving motive"
#        e.g., "She acted in her own best interest."
#    Sense 5 = "a share in a business or financial involvement"
#        e.g., "He bought a controlling interest in the company."
#    Sense 6 = "money paid for the use of money"
#        e.g., "The bank charges 5% interest."

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [4]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/massimo/nltk_data...


True

In [9]:
from nltk.corpus import wordnet as wn

synsets = wn.synsets("interest", pos=wn.NOUN)
print(f"Nombre de synsets pour 'interest' (nom) : {len(synsets)}")
for i, syn in enumerate(synsets):
    print(f"Synset {i+1} : {syn.name()}")
    print(f"Définition : {syn.definition()}")
    print(f"Lemmes : {syn.lemma_names()}")
    print(f"Exemples : {syn.examples()}")
    print(f"=================================")


Nombre de synsets pour 'interest' (nom) : 7
Synset 1 : interest.n.01
Définition : a sense of concern with and curiosity about someone or something
Lemmes : ['interest', 'involvement']
Exemples : ['an interest in music']
Synset 2 : sake.n.01
Définition : a reason for wanting something done
Lemmes : ['sake', 'interest']
Exemples : ['for your sake', 'died for the sake of his country', 'in the interest of safety', 'in the common interest']
Synset 3 : interest.n.03
Définition : the power of attracting or holding one's attention (because it is unusual or exciting etc.)
Lemmes : ['interest', 'interestingness']
Exemples : ['they said nothing of great interest', 'primary colors can add interest to a room']
Synset 4 : interest.n.04
Définition : a fixed charge for borrowing money; usually a percentage of the amount borrowed
Lemmes : ['interest']
Exemples : ['how much interest do you pay on your mortgage?']
Synset 5 : interest.n.05
Définition : (law) a right or legal share of something; a financia

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [ ]:
import nltk
#nltk.download('stopwords')

In [6]:
from random import randrange

In [ ]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

definitions = [
    "readiness to give attention",
    "quality of causing attention to be given to",
    "activity, etc. that one gives attention to",
    "advantage, advancement or favor",
    "a share in a company or business",
    "money paid for the use of money"
]

stop_words = set(stopwords.words("english")) | set(string.punctuation)
senses1 = []
for definition in definitions:
    tokens = word_tokenize(definition.lower())
    filtered = [w for w in tokens if w.isalpha() and len(w) >= 4 and w not in stop_words]
    unique_words = list(set(filtered))
    senses1.append(unique_words)

print(senses1)


[['readiness', 'attention', 'give'], ['given', 'causing', 'attention', 'quality'], ['gives', 'activity', 'attention'], ['favor', 'advantage', 'advancement'], ['share', 'company', 'business'], ['money', 'paid']]


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [8]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = [
    ["attention", "curiosity", "concern", "readiness"],
    ["attention", "attracting", "exciting", "noteworthy", "unusual"],
    ["hobby", "activity", "diversion", "recreation", "pastime"],
    ["benefit", "advantage", "favor", "gain", "profit"],
    ["share", "ownership", "stake", "equity", "business", "company"],
    ["money", "loan", "borrowing", "charge", "percentage", "payment"]
]

print(senses2)


[['attention', 'curiosity', 'concern', 'readiness'], ['attention', 'attracting', 'exciting', 'noteworthy', 'unusual'], ['hobby', 'activity', 'diversion', 'recreation', 'pastime'], ['benefit', 'advantage', 'favor', 'gain', 'profit'], ['share', 'ownership', 'stake', 'equity', 'business', 'company'], ['money', 'loan', 'borrowing', 'charge', 'percentage', 'payment']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [10]:
# Veuillez répondre ici à la question.
with open("interest-original.txt", encoding="utf-8") as f:
    data = f.read()

sentences = [s.strip().split() for s in data.split("$$") if s.strip()]

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
from random import sample
for s in sample(sentences, 3):
    print(" ".join(s))

Il y a 2368 phrases.
En voici 3 au hasard :
====================================== artra group said a three-step recapitalization by its lori corp. unit will cut lori 's annual interest_6 expense by $ 3 million and save lori $ 2 million annually in preferred dividends .
but such partnerships are common in mining , where there are n't problems or conflict of interest_4 or risk of cheating by a partner , '' trelleborg 's president says .
`` there would have to be some cataclysmic event for us not to bring our interest_6 rate down .


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [11]:
# Veuillez répondre ici à la question.
from nltk.corpus import stopwords
import random

def wsd_lesk(senses, sentence, window_size=5):
    stop_words = set(stopwords.words('english'))
    
    # Trouver l'index de "interest" annoté avec un sens
    index_interest = next(i for i, word in enumerate(sentence) if word.startswith("interest_"))
    
    # Nettoyage du contexte (voisinage gauche + droite)
    context = sentence[max(0, index_interest - window_size): index_interest] + \
              sentence[index_interest + 1: index_interest + 1 + window_size]
    
    context_clean = [w.lower() for w in context if w.lower() not in stop_words and len(w) > 3]
    
    # Compter le chevauchement avec chaque sens
    overlaps = []
    for sense_keywords in senses:
        overlap = len(set(context_clean) & set(sense_keywords))
        overlaps.append(overlap)
    
    # Résoudre égalités au hasard
    max_overlap = max(overlaps)
    candidates = [i for i, v in enumerate(overlaps) if v == max_overlap]
    
    return random.choice(candidates) + 1  # retour 1 à 6 (pas 0 à 5)


**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [45]:
# Veuillez répondre ici à la question.
def evaluate_wsd(fct_name, senses, sentences, window_size=8):
    correct = 0
    total = 0
    
    for sentence in sentences:
        # Extraire le vrai sens
        target = next((w for w in sentence if w.startswith("interest_")), None)
        if not target:
            continue
        true_sense = int(target.split("_")[1].split("/")[0])
        
        predicted_sense = fct_name(senses, sentence, window_size)
        
        if predicted_sense == true_sense:
            correct += 1
        total += 1
    
    return 100 * correct / total  # score en pourcentage


**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [13]:
# Veuillez répondre ici à la question.
score1 = evaluate_wsd(wsd_lesk, senses1, sentences)
score2 = evaluate_wsd(wsd_lesk, senses2, sentences)

print(f"Score avec senses1 : {score1:.2f}%")
print(f"Score avec senses2 : {score2:.2f}%")


Score avec senses1 : 18.08%
Score avec senses2 : 22.49%


## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [17]:
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [18]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [19]:
# Veuillez répondre ici à la question.
import numpy as np
from nltk.corpus import stopwords
import random

def wsd_word2vec(senses, sentence, window_size=5):
    stop_words = set(stopwords.words('english'))

    # Localiser le mot annoté interest_X
    idx = next(i for i, word in enumerate(sentence) if word.startswith("interest_"))
    
    # Extraire le contexte (voisinage autour du mot cible)
    context = sentence[max(0, idx - window_size): idx] + sentence[idx + 1: idx + 1 + window_size]
    context = [w.lower() for w in context if w.lower() not in stop_words and len(w) > 3 and w.isalpha()]
    
    # Calculer la similarité moyenne entre le contexte et chaque sens
    similarities = []
    for keywords in senses:
        keywords_vecs = [wv_model[w] for w in keywords if w in wv_model]
        context_vecs = [wv_model[w] for w in context if w in wv_model]

        if not keywords_vecs or not context_vecs:
            similarities.append(0)
            continue

        # Moyenne des vecteurs
        sense_vec = np.mean(keywords_vecs, axis=0)
        context_vec = np.mean(context_vecs, axis=0)

        # Similarité cosinus
        similarity = np.dot(sense_vec, context_vec) / (np.linalg.norm(sense_vec) * np.linalg.norm(context_vec))
        similarities.append(similarity)

    max_sim = max(similarities)
    candidates = [i for i, v in enumerate(similarities) if v == max_sim]
    
    return random.choice(candidates) + 1  # retourne entre 1 et 6


**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [20]:
# Veuillez répondre ici à la question.
score_w2v = evaluate_wsd(wsd_word2vec, senses2, sentences)
print(f"Score avec word2vec et senses2 : {score_w2v:.2f}%")

Score avec word2vec et senses2 : 64.59%


## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [62]:
# Veuillez répondre ici à la question.

items_with_features = []

k = 2

for sentence in sentences:
    
    for i, word in enumerate(sentence):
        if word.startswith('interest'):
            features = {}
            sens = ""
            
            try:
                if '_' in word: # We got a match !
                    (word0, sens) = word.split('_')
                    features['word0'] = word0
                    
                    for depth in range(1,k+1) :
                        if i - depth >= 0:
                            features[f'word-{depth}'] = sentence[i - depth].lower()
                        if i + depth < len(sentence) :
                            features[f'word+{depth}'] = sentence[i + depth].lower()
                        else:
                            features[f'word-{depth}'] = 'NONE'
                        
    
            except Exception as e:
                print(f"Error processing word '{word}' in {sentence}: {e}")
                
            items_with_features.append([features, sens])
            break
        
            
print(f"Nombre total d'instances: {len(items_with_features)}")
print("Exemples d'instances:")
for i in range(min(3, len(items_with_features))):
    print(items_with_features[i])
    


Nombre total d'instances: 2368
Exemples d'instances:
[{'word0': 'interest', 'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': '.'}, '6']
[{'word0': 'interest', 'word-1': 'declining', 'word+1': 'rates', 'word-2': 'indicate', 'word+2': 'because'}, '6']
[{'word0': 'interest', 'word-1': 'short-term', 'word+1': 'rates', 'word-2': 'in', 'word+2': '.'}, '6']


**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [65]:
# Check for empty elements (It can insert issues later on)
items_with_features = [item for item in items_with_features if item[1] != '']
print(f"Cleaned data, new length: {len(items_with_features)}")

Cleaned data, new length: 2367


In [ ]:
from sklearn.model_selection import train_test_split

X = [item[0] for item in items_with_features]
y = [item[1] for item in items_with_features]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

iwf_train = list(zip(X_train, y_train))
iwf_test = list(zip(X_test, y_test))

print(iwf_test[:3])


[({'word0': 'interest', 'word-1': 'short', 'word+1': 'dropped', 'word-2': 'that', 'word+2': 'to'}, '5'), ({'word0': 'interest', 'word-1': '}', 'word+1': 'rates', 'word+2': ','}, '6'), ({'word0': 'interests', 'word-1': 'minority', 'word+1': 'rose', 'word-2': 'to', 'word+2': 'to'}, '5')]


In [67]:
print("Total :", len(items_with_features))
print("Train :", len(iwf_train), "| Test :", len(iwf_test))
print("Sample train:", iwf_train[0])
print("Sample test :", iwf_test[0])

Total : 2367
Train : 1893 | Test : 474
Sample train: ({'word0': 'interest', 'word-1': 'in', 'word+1': 'rates', 'word-2': 'dips', 'word+2': '.'}, '6')
Sample test : ({'word0': 'interest', 'word-1': 'short', 'word+1': 'dropped', 'word-2': 'that', 'word+2': 'to'}, '5')


**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [43]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.

classifier = naivebayes.NaiveBayesClassifier.train(iwf_train)
print("Classifier accuracy (train):", nltk.classify.accuracy(classifier, iwf_train))
print("Classifier accuracy (test):", nltk.classify.accuracy(classifier, iwf_test))

Classifier accuracy (train): 0.9613960867265997
Classifier accuracy (test): 0.89937106918239


**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [34]:
classifier.show_most_informative_features(20)

Most Informative Features
                   word0 = 'interests'         3 : 1      =    143.6 : 1.0
                   word1 = 'in'                5 : 6      =     66.8 : 1.0
                   word1 = 'of'                4 : 6      =     36.7 : 1.0
                  word-1 = 'other'             3 : 6      =     36.6 : 1.0
                   word2 = '.'                 6 : 5      =     26.9 : 1.0
                   word2 = ','                 6 : 1      =     25.3 : 1.0
                  word-2 = 'have'              1 : 6      =     24.1 : 1.0
                  word-1 = 'own'               4 : 6      =     19.9 : 1.0
                  word-1 = 'in'                6 : 5      =     17.5 : 1.0
                   word2 = "''"                4 : 6      =     14.9 : 1.0
                   word2 = 'and'               6 : 1      =     13.9 : 1.0
                   word1 = 'was'               1 : 6      =     12.8 : 1.0
                  word-2 = 'also'              5 : 6      =     11.7 : 1.0

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [32]:
def evaluate_wsd_supervised(classifier, items_with_features):
    total = len(items_with_features)
    correct_by_sense = {}
    total_by_sense = {}

    for features, true_label in items_with_features:
        predicted_label = classifier.classify(features)

        # Compter par sens
        total_by_sense[true_label] = total_by_sense.get(true_label, 0) + 1
        if predicted_label == true_label:
            correct_by_sense[true_label] = correct_by_sense.get(true_label, 0) + 1

    # Afficher précision par sens
    for sense in sorted(total_by_sense.keys()):
        correct = correct_by_sense.get(sense, 0)
        total_sense = total_by_sense[sense]
        print(f"Sense {sense} : {correct}/{total_sense} ({correct / total_sense:.2%})")


evaluate_wsd_supervised(classifier, iwf_test)

Sense 1 : 59/73 (80.82%)
Sense 2 : 0/3 (0.00%)
Sense 3 : 12/14 (85.71%)
Sense 4 : 33/36 (91.67%)
Sense 5 : 83/100 (83.00%)
Sense 6 : 242/251 (96.41%)


## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.